In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from pyvi import ViTokenizer
from transformers import T5ForConditionalGeneration, T5Tokenizer, AutoModel, AutoTokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import torch

In [ ]:
def load_T5():
    t5_model = T5ForConditionalGeneration.from_pretrained("NlpHUST/t5-small-vi-summarization")
    t5_tokenizer = T5Tokenizer.from_pretrained("NlpHUST/t5-small-vi-summarization")
    return t5_model, t5_tokenizer
t5_model, t5_tokenizer = load_T5()

In [ ]:
def summarize(sentence):
    with torch.no_grad():
        tokenized_text = t5_tokenizer.encode(sentence, return_tensors="pt")
        summary_ids = t5_model.generate(tokenized_text, max_length=512, num_beams=5,
                                     repetition_penalty=2.5, length_penalty=1.0, early_stopping=True)
    return t5_tokenizer.decode(summary_ids[0], skip_special_tokens=True)


In [ ]:
def preprocess(text):

    text = re.sub(r'http\S+', '', text)

    text = re.sub(r'[^\w\s\d+]', '', text)

    text = text.lower()

    tokens = ViTokenizer.tokenize(text).split()

    stopwords_file = '/Users/quocminh/Desktop/Đồ Án/chinh thuc/final/stop_word.txt'
    with open(stopwords_file, 'r', encoding='utf-8') as f:
        stop_words = set([line.strip() for line in f])

    tokens = [token for token in tokens if token not in stop_words]

    tokens = [token.replace('\n', ' ') for token in tokens]

    text = ' '.join(tokens)

    return text

In [ ]:
sentence = '''
Tổng cục Hải quan Trung Quốc (GACC) đã thông báo phê duyệt cho 13 cơ sở đóng gói và 70 mã số vùng trồng khoai lang của Việt Nam được phép xuất khẩu vào thị trường Trung Quốc. Ngày 5.4, Cục Bảo vệ thực vật (Bộ NN-PTNT) cho biết, sau gần 1 tháng tiến hành kiểm tra trực tuyến, phía GACC đã gửi công hàm thông báo cấp phép cho 13 cơ sở đóng gói và 70 mã số vùng trồng khoai lang được phép xuất khẩu vào thị trường Trung Quốc. Theo ước tính, diện tích trồng khoai lang được GACC cấp phép trong đợt đầu tiên này lên tới trên 1.000 ha và phân bố ở khắp các địa phương trên cả nước, gồm: Quảng Ninh, Đắk Lắk, Kon Tum, Gia Lai, Vĩnh Long, Đồng Tháp… Dự kiến, Vĩnh Long sẽ là địa phương tổ chức lễ xuất khẩu lô hàng khoai lang đầu tiên của Việt Nam vào thị trường Trung Quốc. Ông Hoàng Trung, Cục trưởng Cục Bảo vệ thực vật, cho rằng thông báo từ GACC rất có ý nghĩa với nông dân Việt Nam khi khoai lang sắp vào vụ thu hoạch. Hiện tại, Cục Bảo vệ thực vật đang tích cực phối hợp với các địa phương trong công tác chuẩn bị cho container khoai lang đầu tiên xuất khẩu sang Trung Quốc trong khoảng 1 tháng nữa. Bên cạnh đó, Cục Bảo vệ thực vật đã có chỉ đạo chi cục, trạm kiểm dịch thực vật ở địa phương theo dõi, kiểm tra, giám sát quy trình canh tác, đóng gói khoai lang tuân thủ các quy định Việt Nam đã cam kết trong nghị định thư ký với Trung Quốc. "Ngành bảo vệ thực vật sẽ áp dụng mọi biện pháp, đảm bảo các lô khoai lang xuất khẩu sang Trung Quốc không nhiễm đối tượng sinh vật gây hại mà nước này yêu cầu. Các lô hàng phải có đầy đủ chứng thư kiểm dịch thực vật, đáp ứng yêu cầu vệ sinh, an toàn thực phẩm của phía Trung Quốc", ông Trung nói. Theo thống kê của Cục Trồng trọt (Bộ NN-PTNT), Việt Nam có khoảng 100.000 ha trồng khoai lang, tổng sản lượng đạt 1,2 - 1,3 triệu tấn. Năng suất khoai lang cao nhất được ghi nhận ở khu vực ĐBSCL, lên tới 25 tấn/ha, cao gấp đôi so với khu vực miền Bắc, chỉ đạt 12 - 13 tấn/ha.
'''

In [ ]:
result = preprocess(summarize(sentence))

In [ ]:
def load_bert():
    v_phobert = AutoModel.from_pretrained("vinai/phobert-base")
    v_tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)
    return v_phobert, v_tokenizer
phobert, tokenizer = load_bert()

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.decoder.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
def phobert_embed_sentence(padded, mask, model=phobert):
    padded = torch.tensor(padded).to(torch.long)
    mask = torch.tensor(mask)
    with torch.no_grad():
        last_hidden_states = model(input_ids=padded, attention_mask=mask)[0]
    vector = last_hidden_states[:, 0, :].numpy()
    return vector.flatten()

In [ ]:
def phobert_embed_data(data, tokenizer=tokenizer):
    MAX_LENGTH = tokenizer.model_max_length

    embedded_data = np.array([])
    for line in data:
        tokenized_line = tokenizer.encode(line, max_length=MAX_LENGTH, truncation=True)

        padded_line = pad_sequences([tokenized_line], maxlen=MAX_LENGTH, padding='post', value=1)

        mask = np.where(padded_line == 1, 0, 1)

        embedded_line = phobert_embed_sentence(padded_line, mask)

        if embedded_data.shape[0] == 0:
            embedded_data = np.empty((0, embedded_line.shape[0]), 'float32')

        embedded_data = np.concatenate((embedded_data, [embedded_line]))
    return embedded_data


In [ ]:
print(result)

cục bảo_vệ thực_vật nnptnt 1 tiến_hành kiểm_tra trực_tuyến gacc gửi công_hàm thông_báo phép 13 cơ_sở đóng_gói 70 mã_số trồng khoai_lang việt nam phép xuất_khẩu thị_trường trung_quốc


In [ ]:
print(phobert_embed_data(result))

[[-0.08201276  0.35448435 -0.47791922 ...  0.19753371  0.02745232
  -0.34429142]
 [-0.07525083 -0.0960936  -0.59531957 ...  0.27698636  0.15752268
  -0.09860051]
 [-0.08201276  0.35448435 -0.47791922 ...  0.19753371  0.02745232
  -0.34429142]
 ...
 [-0.02235891 -0.00309107 -0.19624488 ...  0.17782727 -0.15630639
  -0.49832338]
 [ 0.08797755  0.18068972 -0.3902284  ... -0.12835063  0.7044834
  -0.28385112]
 [-0.08201276  0.35448435 -0.47791922 ...  0.19753371  0.02745232
  -0.34429142]]
